## Extracting a tableau dashboard screenshot while applying multiple filters and creating a presentaion using just a Python code

## Trick & Tips:
The explanation of the basic code is given below. However, in real life sceanario, there might be various Tableau server views with multiple filter combinations needed for creating a presesntation (something that I had to execute for my workspace). For that, please do write codes as multiple functions (functions for creating snapshots, tripmming image, and making presentations seperately) and create iterative code logics that would parse through multiple Tableau Views and filter combinations :)

Here, we begin with connecting our python script to the Tableau Dahsboard by using Tableau APIs
Connecting to the Tableau server through Tableau-Python API

In [ ]:
from tableau_api_lib import TableauServerConnection
from tableau_api_lib.utils.querying import get_views_dataframe, get_view_data_dataframe

This is the credential we are going to use for communicating with your tableau server. 
1. The server name should be your company/public Tableau server name. Replace the xyz below with the whatever that name is. For eg: Salesforce, Google. 
2. The api version can be the latest version of the Tabelau API available. Here, I'm using 3.9
3. The username and password should be associated with the credentials that enable you to view the desired Tableau site.
4. The site name and site url should be the designated Tbaleau site name of your company. You should be able to get that through your URL (the url will contain your site)

In [ ]:
tableau_server_config = {
        'tableau_prod': {
                'server': 'https://tableau.xyz.com',  
                'api_version': '3.9',
                'username': 'Username',
                'password': 'Passowrd',
                'site_name': 'YourSiteName',
                'site_url': 'YourSiteURLName'}}


This can be used to check your connection.
If the connection is active the reponse will be : <Response [200]>

In [ ]:
conn = TableauServerConnection(tableau_server_config)
conn.sign_in()

The below code is used to get all the names of the views present in your server.
The print statement will print all the published Tableau views and their ids present in the Site URL name you mentioned above.
This is a really important step as you may require the View <-> ID mapping in order to capture a particular tableau view.

In [ ]:
import pandas as pd
pd.set_option('display.max_rows', None)

views_df = get_views_dataframe(conn)
print(views_df[['name', 'id']])

### Accessing the Tableau server dashboard view

Below are the parameters of the screen capture we wish to apply. the list is called pdf_params and these are 
the values needed to be passed through the API. 
The filter value is the filter 'Name' you'd want to apply. For example: Country, Sales Division, Rank number etc
I've assigned it as None now, because i'll show you show to pass on a single filter value as well as multiple combination of filters.

In [ ]:
pdf_params = {
            'type': 'type=A4',
            'orientation': 'orientation=Landscape',
            'filter': None }

Here,we're plugging in value for the 'filter' key of the pdf_params dictionary above.
1. The first filter is associated with the filters called 'Region' (This should be present in the dashboard you're viewing). Here, we want the dashboard to filter out on the Region==AMS filter.
2. The second filter is an example of applying two filters the Region as well as Department filter. Here, we want the dashboard to filter out on the Region==AMS and Department==Sales filter 

In [ ]:
pdf_params['filter'] = f'vf_Region={AMS}'
pdf_params['filter'] = f'vf_Region={AMS}&vf_Department={Sales}' 

When you want a screen grab of your newly filter applied dashboard view, write the code below. Here, we need to pass the dashboard_view_id which is the corresponding id of the Tableau server view you want to capture. (use the above list to map and plug in the id value)

In [ ]:
img = conn.query_view_image(view_id=dashboard_view_id, parameter_dict=pdf_params)

The below code is for saving the image screenshot of the applied Tableau server view, in the desired path

In [ ]:
with open(os.path.join(os.path.abspath('./Tableau_Screenshot_Export'),f'export.png'), 'wb') as img_file:
                    img_file.write(img.content)

### Altering the Dashboard Image

We're going to crop the image present in our folder based on our requirements

In [ ]:
# For cropping the image
import os
from PIL import Image

im = Image.open(os.path.join(abs_path,f'{item}-{name}.png'))

#Image dimensions        
width, height = im.size
#print(width,height)
                    
#parameter for 1st
left = 5;top = 140;right = 1420;bottom = 1172
im1 = im.crop((left, top, right, bottom))
        
im1.save(os.path.join(abs_path,f'export_new.png'))

### Creating a presentation

This is a presentation creator. We'll be using the trimmed image above to be placed on the first blank slide on this 5 page bln]ank prsenattion (because we've specified 5 in the code)

In [ ]:
from pptx import Presentation 
from pptx.util import Inches 
import glob


# Creating an Presentation object

ppt = Presentation(os.path.abspath('./Q2 2021 Template.pptx')) 

# Selecting blank slide
blank_slide_layout = ppt.slide_layouts[5] 
slide=ppt.slides[0]
left=Inches(0.5)
top = Inches(0.41)
height= Inches(6.15)
right=0.1
width = Inches(9.1)

img_path = /*plug in the image path*/
pic = slide.shapes.add_picture(img_path, left, top, width, height)

# Save file
ppt.save(f'./Dashboard_Export/ppt/Presentation.pptx')
